In [1]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import os
import numpy as np
BASE_DIR = r"C:\Users\jburd\Desktop\MMASH_1\DataPaper"


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\jburd\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\jburd\AppData\Roaming\Python\Python312\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\jburd\AppData\Roaming\Python\Python312\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_lo

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\jburd\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\jburd\AppData\Roaming\Python\Python312\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\jburd\AppData\Roaming\Python\Python312\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_lo

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



In [26]:
def concatenate(no):    
    user_1_actigraph = pd.read_csv(os.path.join(BASE_DIR, "user_1", "Actigraph.csv"))
    user_1_activity = pd.read_csv(os.path.join(BASE_DIR, "user_1", "Activity.csv"))
    user_1_actigraph.head()
    # print(user_1_actigraph.columns)
    # print(user_1_activity.columns)
    # print(user_1_activity.head(10))
    base=pd.Timestamp("2020-01-01")
    us1=user_1_actigraph
    us1["ts"] = base + pd.to_timedelta(us1["day"] - 1, unit="D") + pd.to_timedelta(us1["time"]) #way to make timestamp unit D is days and default is just time
    #creates a 10s window
    us1["window"]=us1["ts"].dt.floor("10s")

    # print(us1[["day","time","ts","window"]].head(15))
    # print("Unique Windows:",us1["window"].nunique)
    # print(us1.groupby("window").size().describe())
    # Ensure it's sorted by time
    us1 = us1.sort_values("ts").reset_index(drop=True) #sorts and drops the old index to make it non sequential
    us1_1=user_1_activity
    # Build start/end timestamps for intervals
    us1_1["start_ts"] = base + pd.to_timedelta(us1_1["Day"] - 1, unit="D") + pd.to_timedelta(us1_1["Start"] + ":00")
    us1_1["end_ts"]   = base + pd.to_timedelta(us1_1["Day"] - 1, unit="D") + pd.to_timedelta(us1_1["End"] + ":00")

    act1 = us1_1.sort_values("start_ts").reset_index(drop=True)
    # Attach the most recent interval start <= ts, then drop if ts is after end_ts
    us1_labeled = pd.merge_asof(
        us1,
        act1[["start_ts", "end_ts", "Activity"]],
        left_on="ts",
        right_on="start_ts",
        direction="backward"
    )
    us1_labeled["Activity"] = np.where(
        us1_labeled["end_ts"].notna() & (us1_labeled["ts"] <= us1_labeled["end_ts"]),  #if condition met keep Activity if not becomes NaN
        us1_labeled["Activity"],
        np.nan
    )
    MIN_DOM=0.8 #A 10-second window is kept only if at least 80% of its accelerometer rows correspond to the same activity.
    # --- 1) ensure numeric accelerometer columns ---
    feat_cols = ["Axis1", "Axis2", "Axis3", "Vector Magnitude"]
    for c in feat_cols:
        us1_labeled[c]=pd.to_numeric(us1_labeled[c],errors="coerce")
    # --- 2) compute window-level features: mean + std ---
    X_win = us1_labeled.groupby("window")[feat_cols].agg(["mean", "std"])
    X_win.columns = [f"{a}_{b}" for a, b in X_win.columns]
    X_win = X_win.reset_index()


    def dominant_label(s): #s is a column with activity
        s=s.dropna()
        if s.empty:
            return (np.nan,0.0) #return empty list with a dominance of 0
        vc = s.value_counts(normalize=True) #proportions if there a different activity durations
        return (vc.index[0],float(vc.iloc[0]))

    y_win = us1_labeled.groupby("window")["Activity"].apply(dominant_label).reset_index()  #apply the dominant labels
    # print(y_win)
    y_win[["Activity_win", "dominance"]]=pd.DataFrame(y_win["Activity"].to_list(),index=y_win.index)
    y_win = y_win.drop(columns=["Activity"])
    dataset_user1=X_win.merge(y_win,on="window",how="left") #y_win = y_win.drop(columns=["Activity"])

    dataset_user1 = dataset_user1.dropna(subset=["Activity_win"])  
    dataset_user1 = dataset_user1[dataset_user1["dominance"] >= MIN_DOM].reset_index(drop=True) #drops the ones that do not have dominance
    dataset_user1["user"] = "user_1"
    return dataset

# # Quick checks
# print(us1_labeled[["ts", "window", "Activity"]].head(20))
# print(us1_labeled.columns)
# print("Labeled rows:", us1_labeled["Activity"].notna().sum(), "out of", len(us1_labeled))
# print("Unique activities (labeled):", [int(x) for x in sorted(us1_labeled["Activity"].dropna().unique())[:20]])

In [3]:
base=pd.Timestamp("2020-01-01")
us1=user_1_actigraph
us1["ts"] = base + pd.to_timedelta(us1["day"] - 1, unit="D") + pd.to_timedelta(us1["time"]) #way to make timestamp unit D is days and default is just time
#creates a 10s window
us1["window"]=us1["ts"].dt.floor("10s")

print(us1[["day","time","ts","window"]].head(15))
print("Unique Windows:",us1["window"].nunique)
print(us1.groupby("window").size().describe())

    day      time                  ts              window
0     1  10:10:22 2020-01-01 10:10:22 2020-01-01 10:10:20
1     1  10:10:23 2020-01-01 10:10:23 2020-01-01 10:10:20
2     1  10:10:24 2020-01-01 10:10:24 2020-01-01 10:10:20
3     1  10:10:25 2020-01-01 10:10:25 2020-01-01 10:10:20
4     1  10:10:26 2020-01-01 10:10:26 2020-01-01 10:10:20
5     1  10:10:27 2020-01-01 10:10:27 2020-01-01 10:10:20
6     1  10:10:28 2020-01-01 10:10:28 2020-01-01 10:10:20
7     1  10:10:29 2020-01-01 10:10:29 2020-01-01 10:10:20
8     1  10:10:30 2020-01-01 10:10:30 2020-01-01 10:10:30
9     1  10:10:31 2020-01-01 10:10:31 2020-01-01 10:10:30
10    1  10:10:32 2020-01-01 10:10:32 2020-01-01 10:10:30
11    1  10:11:01 2020-01-01 10:11:01 2020-01-01 10:11:00
12    1  10:11:02 2020-01-01 10:11:02 2020-01-01 10:11:00
13    1  10:11:03 2020-01-01 10:11:03 2020-01-01 10:11:00
14    1  10:11:04 2020-01-01 10:11:04 2020-01-01 10:11:00
Unique Windows: <bound method IndexOpsMixin.nunique of 0       2020-01-0

In [4]:
# Ensure it's sorted by time
us1 = us1.sort_values("ts").reset_index(drop=True) #sorts and drops the old index to make it non sequential
us1_1=user_1_activity
# Build start/end timestamps for intervals
us1_1["start_ts"] = base + pd.to_timedelta(us1_1["Day"] - 1, unit="D") + pd.to_timedelta(us1_1["Start"] + ":00")
us1_1["end_ts"]   = base + pd.to_timedelta(us1_1["Day"] - 1, unit="D") + pd.to_timedelta(us1_1["End"] + ":00")

act1 = us1_1.sort_values("start_ts").reset_index(drop=True)
# Attach the most recent interval start <= ts, then drop if ts is after end_ts
us1_labeled = pd.merge_asof(
    us1,
    act1[["start_ts", "end_ts", "Activity"]],
    left_on="ts",
    right_on="start_ts",
    direction="backward"
)
us1_labeled["Activity"] = np.where(
    us1_labeled["end_ts"].notna() & (us1_labeled["ts"] <= us1_labeled["end_ts"]),  #if condition met keep Activity if not becomes NaN
    us1_labeled["Activity"],
    np.nan
)

# Quick checks
print(us1_labeled[["ts", "window", "Activity"]].head(20))
print(us1_labeled.columns)
print("Labeled rows:", us1_labeled["Activity"].notna().sum(), "out of", len(us1_labeled))
print("Unique activities (labeled):", [int(x) for x in sorted(us1_labeled["Activity"].dropna().unique())[:20]])

                    ts              window  Activity
0  2020-01-01 10:10:22 2020-01-01 10:10:20       2.0
1  2020-01-01 10:10:23 2020-01-01 10:10:20       2.0
2  2020-01-01 10:10:24 2020-01-01 10:10:20       2.0
3  2020-01-01 10:10:25 2020-01-01 10:10:20       2.0
4  2020-01-01 10:10:26 2020-01-01 10:10:20       2.0
5  2020-01-01 10:10:27 2020-01-01 10:10:20       2.0
6  2020-01-01 10:10:28 2020-01-01 10:10:20       2.0
7  2020-01-01 10:10:29 2020-01-01 10:10:20       2.0
8  2020-01-01 10:10:30 2020-01-01 10:10:30       2.0
9  2020-01-01 10:10:31 2020-01-01 10:10:30       2.0
10 2020-01-01 10:10:32 2020-01-01 10:10:30       2.0
11 2020-01-01 10:11:01 2020-01-01 10:11:00       2.0
12 2020-01-01 10:11:02 2020-01-01 10:11:00       2.0
13 2020-01-01 10:11:03 2020-01-01 10:11:00       2.0
14 2020-01-01 10:11:04 2020-01-01 10:11:00       2.0
15 2020-01-01 10:11:05 2020-01-01 10:11:00       2.0
16 2020-01-01 10:11:51 2020-01-01 10:11:50       2.0
17 2020-01-01 10:11:52 2020-01-01 10:11:50    

In [25]:
MIN_DOM=0.8 #A 10-second window is kept only if at least 80% of its accelerometer rows correspond to the same activity.
# --- 1) ensure numeric accelerometer columns ---
feat_cols = ["Axis1", "Axis2", "Axis3", "Vector Magnitude"]
for c in feat_cols:
    us1_labeled[c]=pd.to_numeric(us1_labeled[c],errors="coerce")
# --- 2) compute window-level features: mean + std ---
X_win = us1_labeled.groupby("window")[feat_cols].agg(["mean", "std"])
X_win.columns = [f"{a}_{b}" for a, b in X_win.columns]
X_win = X_win.reset_index()


def dominant_label(s): #s is a column with activity
    s=s.dropna()
    if s.empty:
        return (np.nan,0.0) #return empty list with a dominance of 0
    vc = s.value_counts(normalize=True) #proportions if there a different activity durations
    return (vc.index[0],float(vc.iloc[0]))

y_win = us1_labeled.groupby("window")["Activity"].apply(dominant_label).reset_index()  #apply the dominant labels
# print(y_win)
y_win[["Activity_win", "dominance"]]=pd.DataFrame(y_win["Activity"].to_list(),index=y_win.index)
y_win = y_win.drop(columns=["Activity"])
dataset_user1=X_win.merge(y_win,on="window",how="left") #y_win = y_win.drop(columns=["Activity"])

dataset_user1 = dataset_user1.dropna(subset=["Activity_win"])  
dataset_user1 = dataset_user1[dataset_user1["dominance"] >= MIN_DOM].reset_index(drop=True) #drops the ones that do not have dominance
dataset_user1["user"] = "user_1"
print(dataset_user1.head())
print("Windows kept:", len(dataset_user1))
print("Activities:", sorted(dataset_user1["Activity_win"].unique()))
print(dataset_user1["dominance"].describe())

               window  Axis1_mean  Axis1_std  Axis2_mean  Axis2_std  \
0 2020-01-01 10:10:20   20.375000  34.735480   12.750000  11.548036   
1 2020-01-01 10:10:30   35.666667  25.696952   53.333333  17.243356   
2 2020-01-01 10:11:00   43.400000  33.627370   45.000000  29.197603   
3 2020-01-01 10:11:50   25.555556  25.239409   19.333333  24.402869   
4 2020-01-01 10:12:00   47.800000  33.469057   98.400000  81.124322   

   Axis3_mean   Axis3_std  Vector Magnitude_mean  Vector Magnitude_std  \
0   33.250000   31.349413              45.683750             43.143837   
1   68.333333   76.787586              99.130000             72.752297   
2  120.800000   59.440727             140.454000             63.122547   
3   57.000000   54.703748              71.177778             57.754984   
4  164.500000  125.772502             206.190000            140.164040   

   Activity_win  dominance    user  
0           2.0        1.0  user_1  
1           2.0        1.0  user_1  
2           2.0  